In [21]:
# I want to take video file and upload it to Kafka and store metadata in PostgreSQL

from time import sleep
from json import dumps
from json import loads
from kafka import KafkaConsumer

from kafka import KafkaProducer
from kafka import KafkaClient

from pathlib import Path   
import psycopg2

topics = 'remi_logs'
kafka_broker='192.168.1.161:9092'



# json consumer
# consumer = KafkaConsumer(
#     'remi_logs',
#      bootstrap_servers=['192.168.1.161:9092'],
#      auto_offset_reset='latest',
#      enable_auto_commit=True,
#      group_id='my-group',
#      #value_deserializer=lambda x: loads(x.decode('utf-8'))
# )
   

# txt consumer
# consumer=KafkaConsumer(topics, bootstrap_servers=[kafka_broker], auto_offset_reset='latest')
# for msg in consumer:
#     print(msg.value)


# cannot import name 'SimpleProducer' from 'kafka'
# kafka=KafkaClient(kafka_broker) 
# producer = SimpleProducer(kafka)
# producer.send_messages(topics, b'Hello From Kafka-Python Producer')

# json producer
# producer = KafkaProducer(bootstrap_servers=[kafka_broker],
#                          value_serializer=lambda x: 
#                          dumps(x).encode('utf-8'))

# for e in range(1000):
#     data = {'number' : e}
#     producer.send(topics, value=data)
#     sleep(5)

producer = KafkaProducer(bootstrap_servers=[kafka_broker])

# for e in range(1000):
#     data = bytes(e)
#     producer.send(topic=topics, value=data)
#     sleep(1)

# def bytes_from_file(filename, chunksize=8192):
#     with open(filename, "rb") as f:
#         while True:
#             chunk = f.read(chunksize)
#             if chunk:
#                 for b in chunk:
#                     yield b
#             else:
#                 break

# example:
# for b in bytes_from_file('/Users/Remigiusz.Boguszewicz/osobiste/docs/diary/installing_macos.org'):
# #     do_stuff_with(b)
# #    producer.send(topic=topics, value=b)
#     print(type(b))  # <class 'int'>




def kafka_upload(file):
    """
    Will upload file profided as parameter
    Returns start and end offset that marks place in Kafka partition, so the uploaded file 
    can be easily retreived
    """
    offset_start = 0
    offset_end = 0
    with open(file, "rb") as f:
        while (byte := f.read(8192)):    
            #print(type(byte))
            future = producer.send(topic=topics, value=byte)

            try:
                record_metadata = future.get(timeout=10)
                #print(record_metadata)
                if offset_start == 0:                
                    offset_start = record_metadata.offset
            except KafkaError:
                # Decide what to do if produce request failed...
                log.exception()
                pass

        
        offset_end = record_metadata.offset
        
        return offset_start,offset_end
    

    
def pg_metadata_upload(title, offset_start, offset_end):
    print(f"title: {title}")
    print(f"offset_start: {offset_start}")
    print(f"offset_end: {offset_end}")
    

    con = psycopg2.connect(database="celebration", user="celebrate", password="forever", host="192.168.1.162", port="5432")
    print("Database opened successfully")    
    
    cur = con.cursor()
    cur.execute(f"insert into song(song_title,song_added,song_class,song_offset_start,song_offset_end,song_notes) values ('{title}',now(),'testing123', {offset_start},{offset_end},'hi');")
    con.commit()
    print("Record inserted successfully")
    con.close()



In [23]:
file = '/Users/Remigiusz.Boguszewicz/trash/takie-tam/Santabarbara - Niuan - Live session 2018-vuLu5soNWZ4.mp4'
#file = '/Users/Remigiusz.Boguszewicz/osobiste/docs/diary/installing_macos.org'
#file = '/Users/Remigiusz.Boguszewicz/tmp/owca.mp4'
#file = '/Users/Remigiusz.Boguszewicz/tmp/file_list.txt'

offset_start,offset_end = kafka_upload(file)
print(f"offset_start: {offset_start}")
print(f"offset_end: {offset_end}")

pg_metadata_upload(Path(file).name, offset_start, offset_end)

offset_start: 12485
offset_end: 14656
title: Santabarbara - Niuan - Live session 2018-vuLu5soNWZ4.mp4
offset_start: 12485
offset_end: 14656
Database opened successfully
Record inserted successfully


In [28]:
from os import listdir
from os.path import isfile, join
mypath = "/Users/Remigiusz.Boguszewicz/trash/takie-tam/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for onlyfile in onlyfiles:
    file = mypath + onlyfile
    print(f"Uploading to Kafka: {file}")
    offset_start,offset_end = kafka_upload(file)
    print(f"offset_start: {offset_start}")
    print(f"offset_end: {offset_end}")

    pg_metadata_upload(Path(file).name, offset_start, offset_end)

Uploading to Kafka: /Users/Remigiusz.Boguszewicz/trash/takie-tam/Gidge - Quasar-JCw4iLthzlQ.mp4
offset_start: 19606
offset_end: 20537
title: Gidge - Quasar-JCw4iLthzlQ.mp4
offset_start: 19606
offset_end: 20537
Database opened successfully
Record inserted successfully
Uploading to Kafka: /Users/Remigiusz.Boguszewicz/trash/takie-tam/Fatboy Slim - Praise You [Official Video]-ruAi4VBoBSM.mp4
offset_start: 20538
offset_end: 22924
title: Fatboy Slim - Praise You [Official Video]-ruAi4VBoBSM.mp4
offset_start: 20538
offset_end: 22924
Database opened successfully
Record inserted successfully
Uploading to Kafka: /Users/Remigiusz.Boguszewicz/trash/takie-tam/Santabarbara - Niuan - Live session 2018-vuLu5soNWZ4.mp4
offset_start: 22925
offset_end: 25096
title: Santabarbara - Niuan - Live session 2018-vuLu5soNWZ4.mp4
offset_start: 22925
offset_end: 25096
Database opened successfully
Record inserted successfully
Uploading to Kafka: /Users/Remigiusz.Boguszewicz/trash/takie-tam/Second Wind-bxJsAeKmya8.m

offset_start: 143756
offset_end: 144928
title: Mari Boine - Vuoi Vuoi Mu (Vuoi Vuoi Me)-NyB90aNkURU.mp4
offset_start: 143756
offset_end: 144928
Database opened successfully
Record inserted successfully
Uploading to Kafka: /Users/Remigiusz.Boguszewicz/trash/takie-tam/Tangerine Dream - Love On A Real Train (State Azure Cover)-ZRSNy8DcIDk.mp4
offset_start: 144929
offset_end: 147720
title: Tangerine Dream - Love On A Real Train (State Azure Cover)-ZRSNy8DcIDk.mp4
offset_start: 144929
offset_end: 147720
Database opened successfully
Record inserted successfully
Uploading to Kafka: /Users/Remigiusz.Boguszewicz/trash/takie-tam/Tangerine Dream - Ricochet (State Azure Cover) _ Eurorack + DAW-Rgw49PKWaPg.mp4
offset_start: 147721
offset_end: 151380
title: Tangerine Dream - Ricochet (State Azure Cover) _ Eurorack + DAW-Rgw49PKWaPg.mp4
offset_start: 147721
offset_end: 151380
Database opened successfully
Record inserted successfully
Uploading to Kafka: /Users/Remigiusz.Boguszewicz/trash/takie-tam/Wis